# Triage Classification

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset

## Get and Explore Data

In [2]:
df = pd.read_csv("Hospital Triage and Patient History.csv")

In [3]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560486 entries, 0 to 560485
Data columns (total 972 columns):
 #    Column                                              Dtype  
---   ------                                              -----  
 0    dep_name                                            object 
 1    esi                                                 float64
 2    age                                                 float64
 3    gender                                              object 
 4    ethnicity                                           object 
 5    race                                                object 
 6    lang                                                object 
 7    religion                                            object 
 8    maritalstatus                                       object 
 9    employstatus                                        object 
 10   insurance_status                                    object 
 11   disposition             

In [4]:
df.head()

,dep_name,esi,age,gender,ethnicity,race,lang,religion,maritalstatus,employstatus,...,cc_vaginaldischarge,cc_vaginalpain,cc_weakness,cc_wheezing,cc_withdrawal-alcohol,cc_woundcheck,cc_woundinfection,cc_woundre-evaluation,cc_wristinjury,cc_wristpain
0,B,4.0,40.0,Male,Hispanic or Latino,White or Caucasian,English,NaN,Single,Full Time,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,B,4.0,66.0,Male,Hispanic or Latino,Native Hawaiian or Other Pacific Islander,English,Pentecostal,Married,Not Employed,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,B,2.0,66.0,Male,Hispanic or Latino,Native Hawaiian or Other Pacific Islander,English,Pentecostal,Married,Not Employed,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,A,2.0,66.0,Male,Hispanic or Latino,Native Hawaiian or Other Pacific Islander,English,Pentecostal,Married,Not Employed,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,A,3.0,84.0,Female,Hispanic or Latino,Other,Other,Pentecostal,Widowed,Retired,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df.disposition.value_counts()

disposition
Discharge    393848
Admit        166638
Name: count, dtype: int64

## Pytorch

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

torch.set_default_dtype(torch.float64)

cuda


In [7]:
# Create custom dataset
class TriageDataset(torch.utils.data.Dataset):
    
    def __init__(self, file_path = "Hospital Triage and Patient History.csv"):
        df = pd.read_csv(file_path)
        
        # Clean data (convert categorical to numeric, remove department name)
        df.drop(columns=['dep_name'], inplace=True)
        for col in df:
            dt = df[col].dtype 
            if dt == int or dt == float:
                df[col].fillna(0, inplace=True)
            else:
                df[col].fillna("", inplace=True)
        
        categorical = []
        for (key, value) in df.dtypes.items():
            if (value == 'object'):
                df[key] = df[key].astype('category')
                categorical.append(key)
                category_num = f'{key}_num'
                df[category_num] = df[key].cat.codes.astype('float64')
            elif (value == 'int64'):
                df[key] = df[key].astype('float64')
        
        df = df.drop(columns=categorical)
        label_ind = df.columns.get_loc('disposition_num')
        features_ind = [i for i in range(df.shape[1])]
        features_ind.remove(label_ind)
        
        # Set features and label
        self.data = df
        self.features = df.iloc[:, features_ind]
        self.label = df.iloc[:, label_ind]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        x = self.features.iloc[index].values
        y = self.label.iloc[index]
        return x, y

In [37]:
# Create train and test datasets
triage_dataset = TriageDataset()

generator = torch.Generator().manual_seed(42)
train_dataset, test_dataset = torch.utils.data.random_split(triage_dataset, [0.8, 0.2], generator=generator)

# Create dataloaders
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)

In [38]:
class SmallModel(torch.nn.Module):

    def __init__(self):
        super(SmallModel, self).__init__()

        self.linear1 = torch.nn.Linear(970, 300) # 972 - 2 features
        self.activation1 = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(300, 100)
        self.activation2 = torch.nn.ReLU()
        self.linear3 = torch.nn.Linear(100, 30)
        self.activation3 = torch.nn.ReLU()
        self.linear4 = torch.nn.Linear(30, 10)
        self.activation4 = torch.nn.ReLU()
        self.linear5 = torch.nn.Linear(10, 1)

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation1(x)
        x = self.linear2(x)
        x = self.activation2(x)
        x = self.linear3(x)
        x = self.activation3(x)
        x = self.linear4(x)
        return x


In [39]:
small_model = SmallModel().to(device)
# optimizer = torch.optim.Adam(small_model.parameters(), lr= 0.005, weight_decay= 0.001)
# optimizer = torch.optim.Adam(small_model.parameters(), lr= 0.01, weight_decay= 0.005)
optimizer = torch.optim.Adam(small_model.parameters(), lr = 0.001, weight_decay=0)

In [47]:
# Training model
def trainer(model, train_loader, test_loader, num_epochs, optimizer):
    train_accs = []
    test_accs = []
    
    for epoch in range(num_epochs):
        model.train()
        
        for i, (features, labels) in enumerate(train_dataloader):
            labels = labels.type(torch.LongTensor)
            features = features.to(device)
            labels = labels.to(device)

            outputs = small_model(features)
            loss = torch.nn.functional.cross_entropy(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % 1 == 0:
                print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_dataloader)}], Loss: {loss.item():.4f}')

        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for images, labels in train_loader:
                labels = labels.type(torch.LongTensor)
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)

                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
        train_accuracy = correct/total
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Accuracy: {train_accuracy:.4f}')
        train_accs.append(train_accuracy)

        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for images, labels in test_loader:
                labels = labels.type(torch.LongTensor)
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)

                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        test_accuracy = correct/total
        print(f'Epoch [{epoch+1}/{num_epochs}], Test Accuracy: {test_accuracy:.4f}')
        test_accs.append(test_accuracy)
    
    return train_accs, test_accs

In [41]:
train_accs, test_accs = trainer(small_model, train_dataloader, test_dataloader, 1, optimizer)

Epoch [1/1], Step [1/3504], Loss: 2.6212
Epoch [1/1], Step [2/3504], Loss: 2.1728
Epoch [1/1], Step [3/3504], Loss: 1.3842
Epoch [1/1], Step [4/3504], Loss: 1.2248
Epoch [1/1], Step [5/3504], Loss: 1.0694
Epoch [1/1], Step [6/3504], Loss: 0.9944
Epoch [1/1], Step [7/3504], Loss: 1.3315
Epoch [1/1], Step [8/3504], Loss: 0.8157
Epoch [1/1], Step [9/3504], Loss: 2.3966
Epoch [1/1], Step [10/3504], Loss: 0.7969
Epoch [1/1], Step [11/3504], Loss: 0.6869
Epoch [1/1], Step [12/3504], Loss: 0.7931
Epoch [1/1], Step [13/3504], Loss: 0.7105
Epoch [1/1], Step [14/3504], Loss: 0.6695
Epoch [1/1], Step [15/3504], Loss: 0.7308
Epoch [1/1], Step [16/3504], Loss: 0.6474
Epoch [1/1], Step [17/3504], Loss: 0.6508
Epoch [1/1], Step [18/3504], Loss: 0.7155
Epoch [1/1], Step [19/3504], Loss: 0.6535
Epoch [1/1], Step [20/3504], Loss: 0.6344
Epoch [1/1], Step [21/3504], Loss: 0.6652
Epoch [1/1], Step [22/3504], Loss: 0.5740
Epoch [1/1], Step [23/3504], Loss: 0.6306
Epoch [1/1], Step [24/3504], Loss: 0.5899
E

Epoch [1/1], Step [198/3504], Loss: 0.4555
Epoch [1/1], Step [199/3504], Loss: 0.4570
Epoch [1/1], Step [200/3504], Loss: 0.4781
Epoch [1/1], Step [201/3504], Loss: 0.4249
Epoch [1/1], Step [202/3504], Loss: 0.4739
Epoch [1/1], Step [203/3504], Loss: 0.7029
Epoch [1/1], Step [204/3504], Loss: 0.4884
Epoch [1/1], Step [205/3504], Loss: 0.5022
Epoch [1/1], Step [206/3504], Loss: 0.5092
Epoch [1/1], Step [207/3504], Loss: 1.0405
Epoch [1/1], Step [208/3504], Loss: 0.4838
Epoch [1/1], Step [209/3504], Loss: 0.5027
Epoch [1/1], Step [210/3504], Loss: 0.4396
Epoch [1/1], Step [211/3504], Loss: 0.4418
Epoch [1/1], Step [212/3504], Loss: 0.4169
Epoch [1/1], Step [213/3504], Loss: 0.5066
Epoch [1/1], Step [214/3504], Loss: 0.4669
Epoch [1/1], Step [215/3504], Loss: 0.4938
Epoch [1/1], Step [216/3504], Loss: 0.4641
Epoch [1/1], Step [217/3504], Loss: 0.6008
Epoch [1/1], Step [218/3504], Loss: 0.4720
Epoch [1/1], Step [219/3504], Loss: 0.4152
Epoch [1/1], Step [220/3504], Loss: 0.4369
Epoch [1/1]

Epoch [1/1], Step [391/3504], Loss: 0.5444
Epoch [1/1], Step [392/3504], Loss: 0.4265
Epoch [1/1], Step [393/3504], Loss: 0.4710
Epoch [1/1], Step [394/3504], Loss: 0.5018
Epoch [1/1], Step [395/3504], Loss: 0.5162
Epoch [1/1], Step [396/3504], Loss: 0.4332
Epoch [1/1], Step [397/3504], Loss: 0.5098
Epoch [1/1], Step [398/3504], Loss: 0.4802
Epoch [1/1], Step [399/3504], Loss: 0.4385
Epoch [1/1], Step [400/3504], Loss: 0.3997
Epoch [1/1], Step [401/3504], Loss: 0.4402
Epoch [1/1], Step [402/3504], Loss: 0.5228
Epoch [1/1], Step [403/3504], Loss: 0.4977
Epoch [1/1], Step [404/3504], Loss: 0.4868
Epoch [1/1], Step [405/3504], Loss: 0.4234
Epoch [1/1], Step [406/3504], Loss: 0.4195
Epoch [1/1], Step [407/3504], Loss: 0.4548
Epoch [1/1], Step [408/3504], Loss: 0.3440
Epoch [1/1], Step [409/3504], Loss: 0.5010
Epoch [1/1], Step [410/3504], Loss: 0.4782
Epoch [1/1], Step [411/3504], Loss: 0.3787
Epoch [1/1], Step [412/3504], Loss: 0.3593
Epoch [1/1], Step [413/3504], Loss: 0.4431
Epoch [1/1]

Epoch [1/1], Step [583/3504], Loss: 0.5548
Epoch [1/1], Step [584/3504], Loss: 0.4177
Epoch [1/1], Step [585/3504], Loss: 0.4655
Epoch [1/1], Step [586/3504], Loss: 0.3653
Epoch [1/1], Step [587/3504], Loss: 0.3564
Epoch [1/1], Step [588/3504], Loss: 0.5069
Epoch [1/1], Step [589/3504], Loss: 0.4612
Epoch [1/1], Step [590/3504], Loss: 0.3995
Epoch [1/1], Step [591/3504], Loss: 0.3600
Epoch [1/1], Step [592/3504], Loss: 0.4493
Epoch [1/1], Step [593/3504], Loss: 0.4664
Epoch [1/1], Step [594/3504], Loss: 0.4521
Epoch [1/1], Step [595/3504], Loss: 0.4846
Epoch [1/1], Step [596/3504], Loss: 0.4362
Epoch [1/1], Step [597/3504], Loss: 0.4495
Epoch [1/1], Step [598/3504], Loss: 0.4351
Epoch [1/1], Step [599/3504], Loss: 0.4164
Epoch [1/1], Step [600/3504], Loss: 0.4348
Epoch [1/1], Step [601/3504], Loss: 1.2799
Epoch [1/1], Step [602/3504], Loss: 0.4419
Epoch [1/1], Step [603/3504], Loss: 0.4089
Epoch [1/1], Step [604/3504], Loss: 0.3952
Epoch [1/1], Step [605/3504], Loss: 0.4183
Epoch [1/1]

Epoch [1/1], Step [774/3504], Loss: 0.3850
Epoch [1/1], Step [775/3504], Loss: 0.4145
Epoch [1/1], Step [776/3504], Loss: 0.4307
Epoch [1/1], Step [777/3504], Loss: 0.3389
Epoch [1/1], Step [778/3504], Loss: 0.3864
Epoch [1/1], Step [779/3504], Loss: 0.3543
Epoch [1/1], Step [780/3504], Loss: 0.5019
Epoch [1/1], Step [781/3504], Loss: 0.3743
Epoch [1/1], Step [782/3504], Loss: 0.5393
Epoch [1/1], Step [783/3504], Loss: 0.2995
Epoch [1/1], Step [784/3504], Loss: 0.3521
Epoch [1/1], Step [785/3504], Loss: 0.4444
Epoch [1/1], Step [786/3504], Loss: 0.4248
Epoch [1/1], Step [787/3504], Loss: 0.4851
Epoch [1/1], Step [788/3504], Loss: 0.3891
Epoch [1/1], Step [789/3504], Loss: 0.4828
Epoch [1/1], Step [790/3504], Loss: 0.3785
Epoch [1/1], Step [791/3504], Loss: 0.3658
Epoch [1/1], Step [792/3504], Loss: 0.4253
Epoch [1/1], Step [793/3504], Loss: 0.4029
Epoch [1/1], Step [794/3504], Loss: 0.3687
Epoch [1/1], Step [795/3504], Loss: 0.3614
Epoch [1/1], Step [796/3504], Loss: 0.4127
Epoch [1/1]

...

Epoch [1/1], Step [2666/3504], Loss: 0.3573
Epoch [1/1], Step [2667/3504], Loss: 0.3929
Epoch [1/1], Step [2668/3504], Loss: 0.3123
Epoch [1/1], Step [2669/3504], Loss: 0.4900
Epoch [1/1], Step [2670/3504], Loss: 0.3400
Epoch [1/1], Step [2671/3504], Loss: 0.3250
Epoch [1/1], Step [2672/3504], Loss: 0.4354
Epoch [1/1], Step [2673/3504], Loss: 0.4433
Epoch [1/1], Step [2674/3504], Loss: 0.3584
Epoch [1/1], Step [2675/3504], Loss: 0.4177
Epoch [1/1], Step [2676/3504], Loss: 0.3527
Epoch [1/1], Step [2677/3504], Loss: 0.3818
Epoch [1/1], Step [2678/3504], Loss: 0.4246
Epoch [1/1], Step [2679/3504], Loss: 0.3253
Epoch [1/1], Step [2680/3504], Loss: 0.3329
Epoch [1/1], Step [2681/3504], Loss: 0.3331
Epoch [1/1], Step [2682/3504], Loss: 0.2663
Epoch [1/1], Step [2683/3504], Loss: 0.3179
Epoch [1/1], Step [2684/3504], Loss: 0.3196
Epoch [1/1], Step [2685/3504], Loss: 0.3958
Epoch [1/1], Step [2686/3504], Loss: 0.5959
Epoch [1/1], Step [2687/3504], Loss: 0.3964
Epoch [1/1], Step [2688/3504], L

Epoch [1/1], Step [2856/3504], Loss: 0.3784
Epoch [1/1], Step [2857/3504], Loss: 0.3548
Epoch [1/1], Step [2858/3504], Loss: 0.3611
Epoch [1/1], Step [2859/3504], Loss: 0.4089
Epoch [1/1], Step [2860/3504], Loss: 0.3342
Epoch [1/1], Step [2861/3504], Loss: 0.3586
Epoch [1/1], Step [2862/3504], Loss: 0.3930
Epoch [1/1], Step [2863/3504], Loss: 0.2484
Epoch [1/1], Step [2864/3504], Loss: 0.3923
Epoch [1/1], Step [2865/3504], Loss: 0.3978
Epoch [1/1], Step [2866/3504], Loss: 0.3851
Epoch [1/1], Step [2867/3504], Loss: 0.3159
Epoch [1/1], Step [2868/3504], Loss: 0.3434
Epoch [1/1], Step [2869/3504], Loss: 0.3889
Epoch [1/1], Step [2870/3504], Loss: 0.3734
Epoch [1/1], Step [2871/3504], Loss: 0.2653
Epoch [1/1], Step [2872/3504], Loss: 0.4613
Epoch [1/1], Step [2873/3504], Loss: 0.3381
Epoch [1/1], Step [2874/3504], Loss: 0.3316
Epoch [1/1], Step [2875/3504], Loss: 0.4075
Epoch [1/1], Step [2876/3504], Loss: 0.2374
Epoch [1/1], Step [2877/3504], Loss: 0.3560
Epoch [1/1], Step [2878/3504], L

Epoch [1/1], Step [3047/3504], Loss: 0.3803
Epoch [1/1], Step [3048/3504], Loss: 0.3893
Epoch [1/1], Step [3049/3504], Loss: 0.3168
Epoch [1/1], Step [3050/3504], Loss: 0.3104
Epoch [1/1], Step [3051/3504], Loss: 0.3697
Epoch [1/1], Step [3052/3504], Loss: 0.4366
Epoch [1/1], Step [3053/3504], Loss: 0.3170
Epoch [1/1], Step [3054/3504], Loss: 0.3948
Epoch [1/1], Step [3055/3504], Loss: 0.3293
Epoch [1/1], Step [3056/3504], Loss: 0.3317
Epoch [1/1], Step [3057/3504], Loss: 0.3180
Epoch [1/1], Step [3058/3504], Loss: 0.3358
Epoch [1/1], Step [3059/3504], Loss: 0.2569
Epoch [1/1], Step [3060/3504], Loss: 0.3406
Epoch [1/1], Step [3061/3504], Loss: 0.3342
Epoch [1/1], Step [3062/3504], Loss: 0.3236
Epoch [1/1], Step [3063/3504], Loss: 0.4149
Epoch [1/1], Step [3064/3504], Loss: 0.4480
Epoch [1/1], Step [3065/3504], Loss: 0.3692
Epoch [1/1], Step [3066/3504], Loss: 0.4103
Epoch [1/1], Step [3067/3504], Loss: 0.3536
Epoch [1/1], Step [3068/3504], Loss: 0.3660
Epoch [1/1], Step [3069/3504], L

Epoch [1/1], Step [3238/3504], Loss: 0.3373
Epoch [1/1], Step [3239/3504], Loss: 0.3286
Epoch [1/1], Step [3240/3504], Loss: 0.3991
Epoch [1/1], Step [3241/3504], Loss: 0.3840
Epoch [1/1], Step [3242/3504], Loss: 0.2999
Epoch [1/1], Step [3243/3504], Loss: 0.3438
Epoch [1/1], Step [3244/3504], Loss: 0.4189
Epoch [1/1], Step [3245/3504], Loss: 0.4176
Epoch [1/1], Step [3246/3504], Loss: 0.4214
Epoch [1/1], Step [3247/3504], Loss: 0.3691
Epoch [1/1], Step [3248/3504], Loss: 0.4139
Epoch [1/1], Step [3249/3504], Loss: 0.3802
Epoch [1/1], Step [3250/3504], Loss: 0.2689
Epoch [1/1], Step [3251/3504], Loss: 0.3293
Epoch [1/1], Step [3252/3504], Loss: 0.3540
Epoch [1/1], Step [3253/3504], Loss: 0.3726
Epoch [1/1], Step [3254/3504], Loss: 0.4798
Epoch [1/1], Step [3255/3504], Loss: 0.3445
Epoch [1/1], Step [3256/3504], Loss: 0.3225
Epoch [1/1], Step [3257/3504], Loss: 0.3346
Epoch [1/1], Step [3258/3504], Loss: 0.3970
Epoch [1/1], Step [3259/3504], Loss: 0.6679
Epoch [1/1], Step [3260/3504], L

Epoch [1/1], Step [3426/3504], Loss: 0.2972
Epoch [1/1], Step [3427/3504], Loss: 0.3202
Epoch [1/1], Step [3428/3504], Loss: 0.3498
Epoch [1/1], Step [3429/3504], Loss: 0.3712
Epoch [1/1], Step [3430/3504], Loss: 0.3324
Epoch [1/1], Step [3431/3504], Loss: 0.2613
Epoch [1/1], Step [3432/3504], Loss: 0.3652
Epoch [1/1], Step [3433/3504], Loss: 0.2847
Epoch [1/1], Step [3434/3504], Loss: 0.3315
Epoch [1/1], Step [3435/3504], Loss: 0.3712
Epoch [1/1], Step [3436/3504], Loss: 0.4178
Epoch [1/1], Step [3437/3504], Loss: 0.3433
Epoch [1/1], Step [3438/3504], Loss: 0.3848
Epoch [1/1], Step [3439/3504], Loss: 0.3367
Epoch [1/1], Step [3440/3504], Loss: 0.3295
Epoch [1/1], Step [3441/3504], Loss: 0.3810
Epoch [1/1], Step [3442/3504], Loss: 0.4316
Epoch [1/1], Step [3443/3504], Loss: 0.3771
Epoch [1/1], Step [3444/3504], Loss: 0.4287
Epoch [1/1], Step [3445/3504], Loss: 0.4020
Epoch [1/1], Step [3446/3504], Loss: 0.4295
Epoch [1/1], Step [3447/3504], Loss: 0.3514
Epoch [1/1], Step [3448/3504], L

In [54]:
class PaperModel(torch.nn.Module):

    def __init__(self):
        super(PaperModel, self).__init__()

        self.linear1 = torch.nn.Linear(970, 30) # 972 - 2 features
        self.activation1 = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(30, 30)
        self.activation2 = torch.nn.ReLU()
        self.linear3 = torch.nn.Linear(30, 30)
        self.activation3 = torch.nn.ReLU()
        self.linear4 = torch.nn.Linear(30, 1)

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation1(x)
        x = self.linear2(x)
        x = self.activation2(x)
        x = self.linear3(x)
        x = self.activation3(x)
        x = self.linear4(x)
        return x


In [55]:
paper_model = PaperModel().to(device)
optimizer = torch.optim.RMSprop(paper_model.parameters(), lr=1e-3)

In [56]:
train_accs2, test_accs2 = trainer(paper_model, train_dataloader, test_dataloader, 1, optimizer)

Epoch [1/1], Step [1/3504], Loss: 0.3677
Epoch [1/1], Step [2/3504], Loss: 0.3757
Epoch [1/1], Step [3/3504], Loss: 0.3656
Epoch [1/1], Step [4/3504], Loss: 0.3582
Epoch [1/1], Step [5/3504], Loss: 0.3829
Epoch [1/1], Step [6/3504], Loss: 0.4005
Epoch [1/1], Step [7/3504], Loss: 0.3451
Epoch [1/1], Step [8/3504], Loss: 0.3555
Epoch [1/1], Step [9/3504], Loss: 0.2830
Epoch [1/1], Step [10/3504], Loss: 0.3125
Epoch [1/1], Step [11/3504], Loss: 0.3114
Epoch [1/1], Step [12/3504], Loss: 0.3141
Epoch [1/1], Step [13/3504], Loss: 0.3746
Epoch [1/1], Step [14/3504], Loss: 0.3350
Epoch [1/1], Step [15/3504], Loss: 0.3950
Epoch [1/1], Step [16/3504], Loss: 0.2670
Epoch [1/1], Step [17/3504], Loss: 0.3665
Epoch [1/1], Step [18/3504], Loss: 0.3557
Epoch [1/1], Step [19/3504], Loss: 0.2613
Epoch [1/1], Step [20/3504], Loss: 0.3077
Epoch [1/1], Step [21/3504], Loss: 0.2782
Epoch [1/1], Step [22/3504], Loss: 0.3448
Epoch [1/1], Step [23/3504], Loss: 0.3446
Epoch [1/1], Step [24/3504], Loss: 0.2620
E

Epoch [1/1], Step [195/3504], Loss: 0.4074
Epoch [1/1], Step [196/3504], Loss: 0.3066
Epoch [1/1], Step [197/3504], Loss: 0.3047
Epoch [1/1], Step [198/3504], Loss: 0.3605
Epoch [1/1], Step [199/3504], Loss: 0.3825
Epoch [1/1], Step [200/3504], Loss: 0.3680
Epoch [1/1], Step [201/3504], Loss: 0.2729
Epoch [1/1], Step [202/3504], Loss: 0.4001
Epoch [1/1], Step [203/3504], Loss: 0.3674
Epoch [1/1], Step [204/3504], Loss: 0.3708
Epoch [1/1], Step [205/3504], Loss: 0.2657
Epoch [1/1], Step [206/3504], Loss: 0.3518
Epoch [1/1], Step [207/3504], Loss: 0.2676
Epoch [1/1], Step [208/3504], Loss: 0.4277
Epoch [1/1], Step [209/3504], Loss: 0.3229
Epoch [1/1], Step [210/3504], Loss: 0.3430
Epoch [1/1], Step [211/3504], Loss: 0.3001
Epoch [1/1], Step [212/3504], Loss: 0.3484
Epoch [1/1], Step [213/3504], Loss: 0.3831
Epoch [1/1], Step [214/3504], Loss: 0.3156
Epoch [1/1], Step [215/3504], Loss: 0.3050
Epoch [1/1], Step [216/3504], Loss: 0.3305
Epoch [1/1], Step [217/3504], Loss: 0.3757
Epoch [1/1]

Epoch [1/1], Step [388/3504], Loss: 0.3039
Epoch [1/1], Step [389/3504], Loss: 0.3192
Epoch [1/1], Step [390/3504], Loss: 0.3802
Epoch [1/1], Step [391/3504], Loss: 0.3298
Epoch [1/1], Step [392/3504], Loss: 0.3450
Epoch [1/1], Step [393/3504], Loss: 0.2780
Epoch [1/1], Step [394/3504], Loss: 0.4001
Epoch [1/1], Step [395/3504], Loss: 0.3506
Epoch [1/1], Step [396/3504], Loss: 0.2932
Epoch [1/1], Step [397/3504], Loss: 0.3783
Epoch [1/1], Step [398/3504], Loss: 0.3491
Epoch [1/1], Step [399/3504], Loss: 0.3330
Epoch [1/1], Step [400/3504], Loss: 0.3814
Epoch [1/1], Step [401/3504], Loss: 0.3208
Epoch [1/1], Step [402/3504], Loss: 0.3085
Epoch [1/1], Step [403/3504], Loss: 0.3466
Epoch [1/1], Step [404/3504], Loss: 0.2676
Epoch [1/1], Step [405/3504], Loss: 0.3432
Epoch [1/1], Step [406/3504], Loss: 0.3165
Epoch [1/1], Step [407/3504], Loss: 0.3453
Epoch [1/1], Step [408/3504], Loss: 0.2958
Epoch [1/1], Step [409/3504], Loss: 0.3783
Epoch [1/1], Step [410/3504], Loss: 0.4123
Epoch [1/1]

Epoch [1/1], Step [581/3504], Loss: 0.4537
Epoch [1/1], Step [582/3504], Loss: 0.4248
Epoch [1/1], Step [583/3504], Loss: 0.2886
Epoch [1/1], Step [584/3504], Loss: 0.3251
Epoch [1/1], Step [585/3504], Loss: 0.3816
Epoch [1/1], Step [586/3504], Loss: 0.3979
Epoch [1/1], Step [587/3504], Loss: 0.3902
Epoch [1/1], Step [588/3504], Loss: 0.3962
Epoch [1/1], Step [589/3504], Loss: 0.3162
Epoch [1/1], Step [590/3504], Loss: 0.4139
Epoch [1/1], Step [591/3504], Loss: 0.3593
Epoch [1/1], Step [592/3504], Loss: 0.2822
Epoch [1/1], Step [593/3504], Loss: 0.3679
Epoch [1/1], Step [594/3504], Loss: 0.3964
Epoch [1/1], Step [595/3504], Loss: 0.2807
Epoch [1/1], Step [596/3504], Loss: 0.3189
Epoch [1/1], Step [597/3504], Loss: 0.3765
Epoch [1/1], Step [598/3504], Loss: 0.4055
Epoch [1/1], Step [599/3504], Loss: 0.4249
Epoch [1/1], Step [600/3504], Loss: 0.3653
Epoch [1/1], Step [601/3504], Loss: 0.2906
Epoch [1/1], Step [602/3504], Loss: 0.3025
Epoch [1/1], Step [603/3504], Loss: 0.3640
Epoch [1/1]

Epoch [1/1], Step [774/3504], Loss: 0.3995
Epoch [1/1], Step [775/3504], Loss: 0.3562
Epoch [1/1], Step [776/3504], Loss: 0.3932
Epoch [1/1], Step [777/3504], Loss: 0.4215
Epoch [1/1], Step [778/3504], Loss: 0.2925
Epoch [1/1], Step [779/3504], Loss: 0.3331
Epoch [1/1], Step [780/3504], Loss: 0.4662
Epoch [1/1], Step [781/3504], Loss: 0.3484
Epoch [1/1], Step [782/3504], Loss: 0.2957
Epoch [1/1], Step [783/3504], Loss: 0.2655
Epoch [1/1], Step [784/3504], Loss: 0.2808
Epoch [1/1], Step [785/3504], Loss: 0.3983
Epoch [1/1], Step [786/3504], Loss: 0.3286
Epoch [1/1], Step [787/3504], Loss: 0.3588
Epoch [1/1], Step [788/3504], Loss: 0.3664
Epoch [1/1], Step [789/3504], Loss: 0.4006
Epoch [1/1], Step [790/3504], Loss: 0.4005
Epoch [1/1], Step [791/3504], Loss: 0.3640
Epoch [1/1], Step [792/3504], Loss: 0.4021
Epoch [1/1], Step [793/3504], Loss: 0.3165
Epoch [1/1], Step [794/3504], Loss: 0.3320
Epoch [1/1], Step [795/3504], Loss: 0.3384
Epoch [1/1], Step [796/3504], Loss: 0.3655
Epoch [1/1]

...

Epoch [1/1], Step [2665/3504], Loss: 0.3808
Epoch [1/1], Step [2666/3504], Loss: 0.3079
Epoch [1/1], Step [2667/3504], Loss: 0.3445
Epoch [1/1], Step [2668/3504], Loss: 0.3667
Epoch [1/1], Step [2669/3504], Loss: 0.3362
Epoch [1/1], Step [2670/3504], Loss: 0.2759
Epoch [1/1], Step [2671/3504], Loss: 0.3090
Epoch [1/1], Step [2672/3504], Loss: 0.3195
Epoch [1/1], Step [2673/3504], Loss: 0.2852
Epoch [1/1], Step [2674/3504], Loss: 0.3453
Epoch [1/1], Step [2675/3504], Loss: 0.3512
Epoch [1/1], Step [2676/3504], Loss: 0.3481
Epoch [1/1], Step [2677/3504], Loss: 0.3479
Epoch [1/1], Step [2678/3504], Loss: 0.3142
Epoch [1/1], Step [2679/3504], Loss: 0.2576
Epoch [1/1], Step [2680/3504], Loss: 0.3022
Epoch [1/1], Step [2681/3504], Loss: 0.3718
Epoch [1/1], Step [2682/3504], Loss: 0.2481
Epoch [1/1], Step [2683/3504], Loss: 0.3440
Epoch [1/1], Step [2684/3504], Loss: 0.3273
Epoch [1/1], Step [2685/3504], Loss: 0.4585
Epoch [1/1], Step [2686/3504], Loss: 0.2996
Epoch [1/1], Step [2687/3504], L

Epoch [1/1], Step [2855/3504], Loss: 0.2860
Epoch [1/1], Step [2856/3504], Loss: 0.2583
Epoch [1/1], Step [2857/3504], Loss: 0.3026
Epoch [1/1], Step [2858/3504], Loss: 0.3621
Epoch [1/1], Step [2859/3504], Loss: 0.3103
Epoch [1/1], Step [2860/3504], Loss: 0.4038
Epoch [1/1], Step [2861/3504], Loss: 0.3012
Epoch [1/1], Step [2862/3504], Loss: 0.4350
Epoch [1/1], Step [2863/3504], Loss: 0.2910
Epoch [1/1], Step [2864/3504], Loss: 0.4779
Epoch [1/1], Step [2865/3504], Loss: 0.3039
Epoch [1/1], Step [2866/3504], Loss: 0.4061
Epoch [1/1], Step [2867/3504], Loss: 0.3623
Epoch [1/1], Step [2868/3504], Loss: 0.4013
Epoch [1/1], Step [2869/3504], Loss: 0.4308
Epoch [1/1], Step [2870/3504], Loss: 0.3772
Epoch [1/1], Step [2871/3504], Loss: 0.3059
Epoch [1/1], Step [2872/3504], Loss: 0.2855
Epoch [1/1], Step [2873/3504], Loss: 0.3239
Epoch [1/1], Step [2874/3504], Loss: 0.3438
Epoch [1/1], Step [2875/3504], Loss: 0.3855
Epoch [1/1], Step [2876/3504], Loss: 0.2762
Epoch [1/1], Step [2877/3504], L

Epoch [1/1], Step [3042/3504], Loss: 0.3658
Epoch [1/1], Step [3043/3504], Loss: 0.3619
Epoch [1/1], Step [3044/3504], Loss: 0.3949
Epoch [1/1], Step [3045/3504], Loss: 0.2430
Epoch [1/1], Step [3046/3504], Loss: 0.3571
Epoch [1/1], Step [3047/3504], Loss: 0.2799
Epoch [1/1], Step [3048/3504], Loss: 0.4089
Epoch [1/1], Step [3049/3504], Loss: 0.3139
Epoch [1/1], Step [3050/3504], Loss: 0.3171
Epoch [1/1], Step [3051/3504], Loss: 0.3555
Epoch [1/1], Step [3052/3504], Loss: 0.4067
Epoch [1/1], Step [3053/3504], Loss: 0.3787
Epoch [1/1], Step [3054/3504], Loss: 0.3702
Epoch [1/1], Step [3055/3504], Loss: 0.4102
Epoch [1/1], Step [3056/3504], Loss: 0.3155
Epoch [1/1], Step [3057/3504], Loss: 0.3526
Epoch [1/1], Step [3058/3504], Loss: 0.3746
Epoch [1/1], Step [3059/3504], Loss: 0.2989
Epoch [1/1], Step [3060/3504], Loss: 0.3508
Epoch [1/1], Step [3061/3504], Loss: 0.3745
Epoch [1/1], Step [3062/3504], Loss: 0.2764
Epoch [1/1], Step [3063/3504], Loss: 0.2988
Epoch [1/1], Step [3064/3504], L

Epoch [1/1], Step [3232/3504], Loss: 0.3767
Epoch [1/1], Step [3233/3504], Loss: 0.3686
Epoch [1/1], Step [3234/3504], Loss: 0.3345
Epoch [1/1], Step [3235/3504], Loss: 0.2976
Epoch [1/1], Step [3236/3504], Loss: 0.3657
Epoch [1/1], Step [3237/3504], Loss: 0.2873
Epoch [1/1], Step [3238/3504], Loss: 0.2510
Epoch [1/1], Step [3239/3504], Loss: 0.4376
Epoch [1/1], Step [3240/3504], Loss: 0.3469
Epoch [1/1], Step [3241/3504], Loss: 0.3344
Epoch [1/1], Step [3242/3504], Loss: 0.3347
Epoch [1/1], Step [3243/3504], Loss: 0.3183
Epoch [1/1], Step [3244/3504], Loss: 0.2843
Epoch [1/1], Step [3245/3504], Loss: 0.3073
Epoch [1/1], Step [3246/3504], Loss: 0.4021
Epoch [1/1], Step [3247/3504], Loss: 0.3325
Epoch [1/1], Step [3248/3504], Loss: 0.3579
Epoch [1/1], Step [3249/3504], Loss: 0.4036
Epoch [1/1], Step [3250/3504], Loss: 0.2990
Epoch [1/1], Step [3251/3504], Loss: 0.2818
Epoch [1/1], Step [3252/3504], Loss: 0.3494
Epoch [1/1], Step [3253/3504], Loss: 0.2568
Epoch [1/1], Step [3254/3504], L

Epoch [1/1], Step [3421/3504], Loss: 0.3677
Epoch [1/1], Step [3422/3504], Loss: 0.3064
Epoch [1/1], Step [3423/3504], Loss: 0.3467
Epoch [1/1], Step [3424/3504], Loss: 0.2486
Epoch [1/1], Step [3425/3504], Loss: 0.3291
Epoch [1/1], Step [3426/3504], Loss: 0.3319
Epoch [1/1], Step [3427/3504], Loss: 0.3988
Epoch [1/1], Step [3428/3504], Loss: 0.4001
Epoch [1/1], Step [3429/3504], Loss: 0.4043
Epoch [1/1], Step [3430/3504], Loss: 0.3626
Epoch [1/1], Step [3431/3504], Loss: 0.4242
Epoch [1/1], Step [3432/3504], Loss: 0.3481
Epoch [1/1], Step [3433/3504], Loss: 0.3698
Epoch [1/1], Step [3434/3504], Loss: 0.3374
Epoch [1/1], Step [3435/3504], Loss: 0.3982
Epoch [1/1], Step [3436/3504], Loss: 0.3442
Epoch [1/1], Step [3437/3504], Loss: 0.4166
Epoch [1/1], Step [3438/3504], Loss: 0.3628
Epoch [1/1], Step [3439/3504], Loss: 0.3480
Epoch [1/1], Step [3440/3504], Loss: 0.3119
Epoch [1/1], Step [3441/3504], Loss: 0.3623
Epoch [1/1], Step [3442/3504], Loss: 0.3530
Epoch [1/1], Step [3443/3504], L